In [6]:
import numpy as np
import pandas as pd
import pyvo
from astropy.io import ascii
from astropy.table import Table

In [7]:
service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
wise_2mass_jkw2_colorcut = service.submit_job("""
            SELECT TOP 100 ra, dec, j_m_2mass, k_m_2mass, w1mpro, w2mpro, w1snr, w2snr
            FROM allwise_p3as_psd
            WHERE k_m_2mass - w2mpro >= 1.8 - 0.848 * (j_m_2mass - k_m_2mass) AND
            j_m_2mass > 0 AND
            w1snr >= 5 AND
            w2snr >= 5
    """)

wise_2mass_jkw2_colorcut.run()

In [11]:
print(type(wise_2mass_jkw2_colorcut))
print(wise_2mass_jkw2_colorcut.phase)

<class 'pyvo.dal.tap.AsyncTAPJob'>
COMPLETED


In [12]:
table = wise_2mass_jkw2_colorcut.fetch_result()
print(type(table))

<class 'pyvo.dal.tap.TAPResults'>


/home/vibin/anaconda3/envs/dp2/lib/python3.10/site-packages/pyvo/dal/query.py:324: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [14]:
table_astropy = table.to_table()
print(type(table_astropy))

<class 'astropy.table.table.Table'>


In [16]:
table_astropy.write("some_table.csv")

In [58]:
table_vo = table.votable
table_vo.to_xml('wise_2mass_jkw2_colorcut.xml')

<class 'astropy.io.votable.tree.VOTableFile'>
